In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# For mutliple devices (GPUs: 4, 5, 6, 7)
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [3]:
# class Lambda(nn.Module):

#     def __init__(self, fn):
#         super(Lambda, self).__init__()
#         self.fn = fn

#     def forward(self, x):
#         return self.fn(x)

In [7]:
import learn2learn as l2l
for name in l2l.vision.benchmarks.list_tasksets():
    print(name)
    if("tiered-imagenet" == name):
        continue
    tasksets = l2l.vision.benchmarks.get_tasksets(name)

omniglot
Files already downloaded and verified
Files already downloaded and verified
mini-imagenet
tiered-imagenet
fc100
cifarfs


In [8]:
Datasets = [
    "omniglot",]
#             "mini-imagenet", 
#             "fc100", "cifarfs", "tiered-imagenet"]
# WaysAndShots = ["omniglot", "mini-imagenet", "fc100", "cifarfs", "tiered-imagenet"]
#!/usr/bin/env python3

"""
Demonstrates how to:
    * use the MAML wrapper for fast-adaptation,
    * use the benchmark interface to load Omniglot, and
    * sample tasks and split them in adaptation and evaluation sets.
"""

import random
import numpy as np
import torch
import learn2learn as l2l
import matplotlib.pyplot as plt
from torch import nn, optim
import os

In [ ]:
def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    return (predictions == targets).sum().float() / targets.size(0)


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):
    data, labels = batch
    data, labels = data.to(device), labels.to(device)

    # Separate data into adaptation/evalutation sets
    adaptation_indices = np.zeros(data.size(0), dtype=bool)
    adaptation_indices[np.arange(shots*ways) * 2] = True
    evaluation_indices = torch.from_numpy(~adaptation_indices)
    adaptation_indices = torch.from_numpy(adaptation_indices)
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        learner.adapt(train_error)

    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    return valid_error, valid_accuracy


def perform_experiment(
        dataset,
        ways=5,
        shots=1,
        meta_lr=0.003,
        fast_lr=0.5,
        meta_batch_size=32,
        adaptation_steps=1,
        num_iterations=64,#60000,
        cuda=True,
        seed=42,
        
):
    print("############### {0} ###############".format(dataset))
    Meta_Train_Accuracy = []
    Meta_Train_Error = []
    Meta_Val_Accuracy = []
    Meta_Val_Error = []
#     Iterations = []
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    device = torch.device('cpu')
    if cuda:
        torch.cuda.manual_seed(seed)
        device = torch.device('cuda')

    # Load train/validation/test tasksets using the benchmark interface
    tasksets = l2l.vision.benchmarks.get_tasksets(dataset,
                                                  train_ways=ways,
                                                  train_samples=2*shots,
                                                  test_ways=ways,
                                                  test_samples=2*shots,
                                                  num_tasks=20000,
                                                  root='~/data',
    )

    #     "omniglot",
#             "mini-imagenet", 
#             "fc100", "cifarfs", "tiered-imagenet"
    # Create model
    if(dataset == "omniglot"):
        model = l2l.vision.models.OmniglotFC(28 ** 2, ways)
    elif(dataset == "mini-imagenet"):
        model = l2l.vision.models.MiniImagenetCNN(ways)
    elif(dataset == "fc100"):
        features = l2l.vision.models.ConvBase(output_size=64, channels=3, max_pool=True)
        features = torch.nn.Sequential(features, Lambda(lambda x: x.view(-1, 256)))
        features.to(device)
        model = torch.nn.Linear(256, ways)
#     elif(dataset = ""):
    else:
        print("Error occured.")
        
    
    model.to(device)
    maml = l2l.algorithms.MAML(model, lr=fast_lr, first_order=False)
    opt = optim.Adam(maml.parameters(), meta_lr)
    loss = nn.CrossEntropyLoss(reduction='mean')

    for iteration in range(num_iterations):
        opt.zero_grad()
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        for task in range(meta_batch_size):
            # Compute meta-training loss
            learner = maml.clone()
            batch = tasksets.train.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            evaluation_error.backward()
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()

            # Compute meta-validation loss
            learner = maml.clone()
            batch = tasksets.validation.sample()
            evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
            meta_valid_error += evaluation_error.item()
            meta_valid_accuracy += evaluation_accuracy.item()

        # Print some metrics
        
        
        meta_train_error =  meta_train_error / meta_batch_size
        meta_train_accuracy = meta_train_accuracy / meta_batch_size
        meta_val_error =  meta_valid_error / meta_batch_size
        meta_val_accuracy = meta_valid_accuracy / meta_batch_size
        if(iteration % 32 ==0):
            print('\n')
            print('Iteration', iteration)
            print('Meta Train Error', meta_train_error)
            print('Meta Train Accuracy', meta_train_accuracy)
            print('Meta Valid Error', meta_val_error)
            print('Meta Valid Accuracy', meta_val_accuracy)

        Meta_Train_Accuracy.append(meta_train_accuracy)
        Meta_Train_Error.append(meta_train_error)
        Meta_Val_Accuracy.append(meta_val_accuracy)
        Meta_Val_Error.append(meta_val_error)

        # Average the accumulated gradients and optimize
        for p in maml.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        opt.step()

    meta_test_error = 0.0
    meta_test_accuracy = 0.0
    for task in range(meta_batch_size):
        # Compute meta-testing loss
        learner = maml.clone()
        batch = tasksets.test.sample()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           shots,
                                                           ways,
                                                           device)
        meta_test_error += evaluation_error.item()
        meta_test_accuracy += evaluation_accuracy.item()
    Meta_Test_Error = meta_test_error / meta_batch_size
    print('Meta Test Error', Meta_Test_Error)
    Meta_Test_Accuracy = meta_test_accuracy / meta_batch_size
    print('Meta Test Accuracy', Meta_Test_Accuracy)
    
    if not os.path.exists('plots'):
        os.makedirs('plots')
    if not os.path.exists('plots/acc'):
        os.makedirs('plots/acc')
    if not os.path.exists('plots/loss'):
        os.makedirs('plots/loss')
    ###### Plot Accuracies ######
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy, label="Meta Train ")
    ax.plot(list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label="Meta Val")
#     ax.text((len(Meta_Val_Accuracy)/2), 0, 'Meta Test:{0}'.format(round(meta_test_accuracy, 2)), style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5})
    # place a text box in upper left in axes coords
    ax.text(0.05, 0.5, 'Meta Test:{0}'.format(round(Meta_Test_Accuracy, 2)), transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
    plt.title('Adaption')
    plt.xlabel('Iteration')
    plt.ylabel('Meta Accuracy')
    ax.legend()
    plt.savefig('./plots/acc/{0}_ways_{1}_shots_{2}_Acc_I_{3}.png'.format(dataset, ways, shots, iteration), 
                dpi=150)
    ###### Plot Accuracies ######

    ###### Plot Errors ######
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(list(range(0, len(Meta_Train_Error))), Meta_Train_Error, label="Meta Train ")
    ax.plot(list(range(0, len(Meta_Val_Error))), Meta_Val_Accuracy, label="Meta Val Lss")
#     ax.text((len(Meta_Val_Accuracy)/2),0, 'Meta Test Accuracy:{0}  Meta Test Loss:{1}'.format(round(meta_test_accuracy, 2),
#                               round(meta_test_error, 2)), style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5})
    ax.text(0.05, 0.5, 'MTestAcc:{0}  MTestLoss:{1}'.format(round(Meta_Test_Accuracy, 2),
                                           round(Meta_Test_Error, 2)), transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
    plt.title('Adaption')
    plt.xlabel('Iteration')
    plt.ylabel('Meta Loss')
    ax.legend()
    plt.savefig('./plots/loss/{0}_ways_{1}_shots_{2}_Loss_I_{3}.png'.format(dataset, ways, shots, iteration),
               dpi=150)
    ###### Plot Errors ######


if __name__ == '__main__':
    Ways = [5]
    Shots = [1, 5]
    Iterations = [4, 10000, 60000, 120000]
    for ways in Ways:
        for shots in Shots:
            for iteration in Iterations:
                for dataset in Datasets:
                    perform_experiment(dataset=dataset,
                        ways=ways,
                        shots=shots,
                        meta_lr=0.003,
                        fast_lr=0.5,
                        meta_batch_size=32,
                        adaptation_steps=1,
                        num_iterations=iteration,
                        cuda=True,
                        seed=42)
    

############### omniglot ###############
Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 1.5371868796646595
Meta Train Accuracy 0.33750000642612576
Meta Valid Error 1.5791204795241356
Meta Valid Accuracy 0.3375000092200935
Meta Test Error 1.381746657192707
Meta Test Accuracy 0.46250001108273864
############### omniglot ###############
Files already downloaded and verified
Files already downloaded and verified


Iteration 0
Meta Train Error 1.5371868796646595
Meta Train Accuracy 0.33750000642612576
Meta Valid Error 1.5791204795241356
Meta Valid Accuracy 0.3375000092200935


Iteration 32
Meta Train Error 0.9426178112626076
Meta Train Accuracy 0.656250013038516
Meta Valid Error 1.0846532871946692
Meta Valid Accuracy 0.5562500106170774


Iteration 64
Meta Train Error 0.9493302307091653
Meta Train Accuracy 0.6437500105239451
Meta Valid Error 0.8882915452122688
Meta Valid Accuracy 0.6937500108033419


Iteration 96
Meta Train Error 0.



Iteration 1536
Meta Train Error 0.3254993657465093
Meta Train Accuracy 0.8875000057742
Meta Valid Error 0.43394802405964583
Meta Valid Accuracy 0.8750000055879354


Iteration 1568
Meta Train Error 0.51693530115881
Meta Train Accuracy 0.8312500067986548
Meta Valid Error 0.7231497026514262
Meta Valid Accuracy 0.7562500089406967


Iteration 1600
Meta Train Error 0.40623748500365764
Meta Train Accuracy 0.8625000072643161
Meta Valid Error 0.6256127159576863
Meta Valid Accuracy 0.7437500106170774


Iteration 1632
Meta Train Error 0.43278237350750715
Meta Train Accuracy 0.8500000070780516
Meta Valid Error 0.4870003488322254
Meta Valid Accuracy 0.8125000093132257


Iteration 1664
Meta Train Error 0.42622897314140573
Meta Train Accuracy 0.8625000072643161
Meta Valid Error 0.5699450887041166
Meta Valid Accuracy 0.8125000060535967


Iteration 1696
Meta Train Error 0.4427071931422688
Meta Train Accuracy 0.8437500083819032
Meta Valid Error 0.44559676220524125
Meta Valid Accuracy 0.875000005587935



Iteration 3136
Meta Train Error 0.41517136280890554
Meta Train Accuracy 0.8562500053085387
Meta Valid Error 0.5155008672154509
Meta Valid Accuracy 0.8312500077299774


Iteration 3168
Meta Train Error 0.233433313871501
Meta Train Accuracy 0.9500000011175871
Meta Valid Error 0.3975758515589405
Meta Valid Accuracy 0.8687500064261258


Iteration 3200
Meta Train Error 0.2795498344639782
Meta Train Accuracy 0.9125000052154064
Meta Valid Error 0.5736335143446922
Meta Valid Accuracy 0.8125000074505806


Iteration 3232
Meta Train Error 0.2644476175773889
Meta Train Accuracy 0.9312500040978193
Meta Valid Error 0.46433524510939606
Meta Valid Accuracy 0.843750006519258


Iteration 3264
Meta Train Error 0.30870539986062795
Meta Train Accuracy 0.8875000057742
Meta Valid Error 0.5262017427303363
Meta Valid Accuracy 0.8125000088475645


Iteration 3296
Meta Train Error 0.30973853400791995
Meta Train Accuracy 0.9312500022351742
Meta Valid Error 0.5077683805575361
Meta Valid Accuracy 0.8250000067055225



Iteration 4736
Meta Train Error 0.21218978241086006
Meta Train Accuracy 0.9500000029802322
Meta Valid Error 0.26880524259468075
Meta Valid Accuracy 0.8875000057742


Iteration 4768
Meta Train Error 0.23597496531147044
Meta Train Accuracy 0.9375000037252903
Meta Valid Error 0.6533976563732722
Meta Valid Accuracy 0.787500009406358


Iteration 4800
Meta Train Error 0.19561920067644678
Meta Train Accuracy 0.9312500040978193
Meta Valid Error 0.6590721085085534
Meta Valid Accuracy 0.7812500093132257


Iteration 4832
Meta Train Error 0.29948308202438056
Meta Train Accuracy 0.8937500044703484
Meta Valid Error 0.6641878107766388
Meta Valid Accuracy 0.8000000081956387


Iteration 4864
Meta Train Error 0.29043986618489726
Meta Train Accuracy 0.9187500048428774
Meta Valid Error 0.5081928385334322
Meta Valid Accuracy 0.8125000111758709


Iteration 4896
Meta Train Error 0.1252088334513246
Meta Train Accuracy 0.9625000022351742
Meta Valid Error 0.705154339200817
Meta Valid Accuracy 0.76875001098960



Iteration 6304
Meta Train Error 0.14641716786718462
Meta Train Accuracy 0.9562500026077032
Meta Valid Error 0.6230887304845965
Meta Valid Accuracy 0.8062500078231096


Iteration 6336
Meta Train Error 0.24927391204982996
Meta Train Accuracy 0.9125000052154064
Meta Valid Error 0.8133481474869768
Meta Valid Accuracy 0.7562500080093741


Iteration 6368
Meta Train Error 0.19132798744522006
Meta Train Accuracy 0.9500000029802322
Meta Valid Error 0.6517352898372337
Meta Valid Accuracy 0.8250000062398612


Iteration 6400
Meta Train Error 0.28553630185342627
Meta Train Accuracy 0.9250000044703484
Meta Valid Error 0.7054258341522655
Meta Valid Accuracy 0.8000000100582838


Iteration 6432
Meta Train Error 0.23848809899936896
Meta Train Accuracy 0.9437500033527613
Meta Valid Error 0.6438021992762515
Meta Valid Accuracy 0.8187500070780516


Iteration 6464
Meta Train Error 0.25536152040513116
Meta Train Accuracy 0.9125000052154064
Meta Valid Error 0.6260289126221323
Meta Valid Accuracy 0.837500006

...



Iteration 25952
Meta Train Error 0.007087225315984824
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.09073279091558106
Meta Valid Accuracy 0.9762499872595072


Iteration 25984
Meta Train Error 0.018680908470578572
Meta Train Accuracy 0.9937499966472387
Meta Valid Error 0.13945246812249934
Meta Valid Accuracy 0.9687499906867743


Iteration 26016
Meta Train Error 0.026306422145960795
Meta Train Accuracy 0.9937499966472387
Meta Valid Error 0.17731361903960874
Meta Valid Accuracy 0.9649999849498272


Iteration 26048
Meta Train Error 0.0017349568313136388
Meta Train Accuracy 1.0
Meta Valid Error 0.17188565438478065
Meta Valid Accuracy 0.9587499853223562


Iteration 26080
Meta Train Error 0.00663218138961863
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.0982278842528359
Meta Valid Accuracy 0.9812499918043613


Iteration 26112
Meta Train Error 0.005906586121803059
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.1638778051533336
Meta Valid Accuracy 0.96124998



Iteration 27520
Meta Train Error 0.002117195416523998
Meta Train Accuracy 1.0
Meta Valid Error 0.23359009403475284
Meta Valid Accuracy 0.9462499842047691


Iteration 27552
Meta Train Error 0.0042658294535242725
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.22224566348268127
Meta Valid Accuracy 0.9624999891966581


Iteration 27584
Meta Train Error 0.009121728617280667
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.20379377934857246
Meta Valid Accuracy 0.9674999844282866


Iteration 27616
Meta Train Error 0.010215898243387755
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.19221941883452587
Meta Valid Accuracy 0.9637499861419201


Iteration 27648
Meta Train Error 0.008978916154749328
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.21172542683462936
Meta Valid Accuracy 0.9612499848008156


Iteration 27680
Meta Train Error 0.004905549581238233
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.2021474780843846
Meta Valid Accuracy 0.961249



Iteration 29088
Meta Train Error 0.008965412744082357
Meta Train Accuracy 0.994999997317791
Meta Valid Error 0.2232038891912964
Meta Valid Accuracy 0.9562499839812517


Iteration 29120
Meta Train Error 0.005693960485713578
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.19062196271552523
Meta Valid Accuracy 0.9749999903142452


Iteration 29152
Meta Train Error 0.006275897480723103
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.10179965142415703
Meta Valid Accuracy 0.9712499864399433


Iteration 29184
Meta Train Error 0.012420060176452807
Meta Train Accuracy 0.9962499979883432
Meta Valid Error 0.18080807052905357
Meta Valid Accuracy 0.9649999868124723


Iteration 29216
Meta Train Error 0.004635730152472206
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.1748396273839461
Meta Valid Accuracy 0.9712499883025885


Iteration 29248
Meta Train Error 0.006113222600884427
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.19717737435502158
Meta Valid Accur



Iteration 30656
Meta Train Error 0.00840575130810528
Meta Train Accuracy 0.9962499979883432
Meta Valid Error 0.1863385881696118
Meta Valid Accuracy 0.9699999876320362


Iteration 30688
Meta Train Error 0.00640275976574145
Meta Train Accuracy 0.9962499979883432
Meta Valid Error 0.10680575394081693
Meta Valid Accuracy 0.9712499864399433


Iteration 30720
Meta Train Error 0.0025171184180976525
Meta Train Accuracy 1.0
Meta Valid Error 0.13225494221100575
Meta Valid Accuracy 0.9599999897181988


Iteration 30752
Meta Train Error 0.011428078331245928
Meta Train Accuracy 0.9937499966472387
Meta Valid Error 0.08939083375496892
Meta Valid Accuracy 0.9787499886006117


Iteration 30784
Meta Train Error 0.008000887015143121
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.19311733779054285
Meta Valid Accuracy 0.9587499834597111


Iteration 30816
Meta Train Error 0.007074981015549042
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.192887632617726
Meta Valid Accuracy 0.9549999851



Iteration 32224
Meta Train Error 0.002827058713323005
Meta Train Accuracy 0.9987499993294477
Meta Valid Error 0.13632637021320404
Meta Valid Accuracy 0.978749992325902


Iteration 32256
Meta Train Error 0.008799770696370501
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.19872631915345096
Meta Valid Accuracy 0.9662499874830246


Iteration 32288
Meta Train Error 0.008254765700192124
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.2228902130869983
Meta Valid Accuracy 0.9637499861419201


Iteration 32320
Meta Train Error 0.00847432466181175
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.13167352918636865
Meta Valid Accuracy 0.9724999908357859


Iteration 32352
Meta Train Error 0.008742129499829332
Meta Train Accuracy 0.9974999986588955
Meta Valid Error 0.19045400659884137
Meta Valid Accuracy 0.9624999836087227


In [ ]:
# round(12.54673, 2)

In [ ]:


# fig = plt.figure()
# ax = plt.subplot()
# ax.plot(list(range(0, (10))), list(range(0, (10))), label="Meta Train")
# ax.text(5, 0, 'Meta Test:95%', style='italic',
#         bbox={'facecolor': 'red', 'alpha': 0.25, 'pad': 5}, ha='center', va='center')
# # ax.plot(list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label="Meta Val")
# plt.title('Adaption')
# plt.xlabel('Iteration')
# plt.ylabel('Meta Accuracy')
# ax.legend()
# # plt.show()



# # plt.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy,
# #         list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label = ("A", "B"))#, list(range(0, len(Meta_Train_Error))), 
# #          Meta_Train_Error)#,label = ["Meta Train Accuracy", "Meta Train Error"])
# plt.savefig('foo.png')

In [ ]:
# plt.plot(list(range(0, len(Meta_Train_Accuracy))), Meta_Train_Accuracy,
#         list(range(0, len(Meta_Val_Accuracy))), Meta_Val_Accuracy, label = ("A", "B"))#, list(range(0, len(Meta_Train_Error))), 
# #          Meta_Train_Error)#,label = ["Meta Train Accuracy", "Meta Train Error"])

In [32]:
#Define the network
linear = l2l.algorithms.MAML(nn.Linear(20, 10), lr=0.01)
#Clone the parameters
clone = linear.clone()
#Compute loss with cloned network
error = loss(clone(X), y)
#Adapt clone network
clone.adapt(error)
#Meta loss
error = loss(clone(X), y)
#Back-propage the meta loss
error.backward()

NameError: name 'loss' is not defined

In [30]:
import torch
from torch import nn

In [31]:
print(nn.Parameter(torch.zeros([2,2]), requires_grad=True))
# nn.Parameter()

Parameter containing:
tensor([[0., 0.],
        [0., 0.]], requires_grad=True)


In [18]:
tn = torch.randn((3,3))
print(tn)

tensor([[ 0.1364,  0.4121,  0.9692],
        [-0.9406, -0.5043, -0.3111],
        [ 0.8967, -1.3596, -0.4991]])


In [23]:
print(torch.diagonal(tn, -2))

tensor([0.8967])


In [25]:
# help(torch.diagonal)